의사결정나무처럼 Random Forest 도 two moon dataset 을 이용하여 하이퍼 패러매터에 따른 경계면의 변화를 살펴봅니다.

In [1]:
from bokeh.plotting import output_notebook, show
from ensemble_utils import scatterplot_2class
from ensemble_utils import draw_activate_image
from soydata.data.classification import make_moons

import numpy as np
np.set_printoptions(precision=5, suppress=True)

import warnings
warnings.filterwarnings('ignore')

output_notebook()

X, labels = make_moons(n_samples=500, xy_ratio=2.0, x_gap=-0.2, y_gap=-0.15, noise=0.1, seed=0)
p = scatterplot_2class(X, labels, height=400, width=400)
show(p)

Loading BokehJS ...

아래 함수는 의사결정나무의 튜토리얼과 같습니다.

In [2]:
from sklearn.tree import DecisionTreeClassifier

def prepare_elements(model, X, labels):
    score = model.predict_proba(X)
    score = score[:,1] - score[:,0]
    pred = model.predict(X)    
    accuracy = (pred == labels).sum() / labels.shape[0]
    return score, accuracy

하나의 의사결정나무를 과적합이 되도록 학습하였습니다.

In [3]:
dt = DecisionTreeClassifier()
dt.fit(X, labels)

score, accuracy = prepare_elements(dt, X, labels)
title = f'Decision Tree. accuracy={accuracy:.4}'
p = draw_activate_image(dt, X, use_score=True, resolution=100, title=title, height=400, width=400)
p = scatterplot_2class(X, labels, score=score, p=p)
show(p)

Scikit-learn 의 Random Forest Classifier 는 Decision Tree Classifier 를 base estimator 로 이용합니다. 그렇기 때문에 decision tree 에 이용하는 하이퍼 패러매터를 모두 제공합니다. 여기에 `n_estimators` 와 `oob_score` 가 추가됩니다. `n_estimators` 는 학습에 이용할 decision tree 의 개수이며, `oob_score=True` 로 설정하면 학습 과정에서 out of bag error 를 계산합니다.

High bias, low variance 인 간단한 decision trees 를 이용하여 Random Forest 를 학습하더라도 어느 정도 분류가 일어나지만, prediction score 가 -1 이나 1 로 정확히 결정되지 않는 영역이 많습니다. Weak models 를 이용하여 random forest 를 학습하면 여전히 under fitting 이 될 가능성이 높습니다. Random Forest 는 여러 개의 과적합된 low bias 모델들의 예측 결과를 종합하여 variance 를 줄이는 모델입니다.

각각의 decision trees 가 과적합이 되도록 max depth 에 제한을 두지 않은 경우에는 경계면이 잘 학습될 뿐 아니라, 데이터가 존재하지 않은 영역에서는 prediction score 의 절대값이 매우 작음을 확인할 수 있습니다. 그 부분들은 각각의 base estimator 가 임의로 prediction 을 해도 상관이 없기 때문에 각 base estimator 마다 서로 다르게 예측을 하기 때문입니다.

In [4]:
from sklearn.ensemble import RandomForestClassifier
from bokeh.layouts import gridplot
from bokeh.io import save

for depth in [4, None]:
    figures = []

    rf = RandomForestClassifier(
        n_estimators = 100,
        max_depth = depth,
        oob_score = True,
        bootstrap = True
    )
    rf.fit(X, labels)

    score, accuracy = prepare_elements(rf, X, labels)
    title = f'Random Forest (max depth={depth}). accuracy={accuracy:.4}'
    p = draw_activate_image(rf, X, use_score=True, resolution=100, height=400, width=400, title=title)
    p = scatterplot_2class(X, labels, score=score, p=p)
    figures.append(p)

    for dt in rf.estimators_[:5]:
        score, accuracy = prepare_elements(dt, X, labels)
        title = f'Decision Tree. accuracy={accuracy:.4}'
        p = draw_activate_image(dt, X, use_score=True, resolution=100, height=400, width=400, title=title)
        p = scatterplot_2class(X, labels, score=score, p=p)
        figures.append(p)

    gp = gridplot([figures[:3], figures[3:]])
    show(gp)
#     save(gp, f'./figures/random_forest_moon_depth_{depth}.html')

앞서 계산한 out of bag error 에 의한 training accuracy 입니다.

In [5]:
rf.oob_score_

0.97

또한 각 data point, Xi 에 대하여 out of bag score 를 누적하여 저장한 값은 `oob_decision_function_` 에 저장되어 있습니다.

In [6]:
rf.oob_decision_function_.shape

(500, 2)

Feature importances 역시 각 decision tree 의 변수 별 information gain 의 누적 평균입니다.

In [7]:
rf.feature_importances_

array([0.46253, 0.53747])

학습데이터에 대하여 예측 확률을 더 깊게 살펴봅니다. 예측 확률이 (0.49, 0.51) 이면 accuracy 가 높더라도 안전한 예측이 아닙니다.

In [8]:
prob = rf.predict_proba(X)
prob[:5]

array([[0.65, 0.35],
       [0.93, 0.07],
       [0.61, 0.39],
       [0.88, 0.12],
       [0.94, 0.06]])

이를 확인하는 간단한 방법은 각 데이터의 예측 확률의 최대값의 평균을 취하는 것입니다.

In [9]:
max_prob = prob.max(axis=1)
print(max_prob.shape)
print(max_prob.mean())

(500,)
0.97612


In [10]:
print(rf.estimators_[0])
print(f'num base estimators = {len(rf.estimators_)}')

for i, base in enumerate(rf.estimators_[:3]):
    _, accuracy = prepare_elements(base, X, labels)
    print(f'accuracy of one {i}th estimator = {accuracy:4}')

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=2115681183, splitter='best')
num base estimators = 100
accuracy of one 0th estimator = 0.978
accuracy of one 1th estimator = 0.986
accuracy of one 2th estimator = 0.97


미리 만들어둔 `draw_activate_image` 함수를 이용하여 각 영역의 p(1-p) 로 색을 칠해봅니다. 예측확률이 모호하면 p(1-p) 의 값이 커집니다. 이를 통하여 예측이 어려운 부분들을 살펴볼 수 있습니다. 저장된 그림을 살펴보면 데이터가 존재하지 않거나 여러 클래스의 데이터가 겹쳐진 경계부분에서 이 값이 높음을 확인할 수 있습니다.

In [11]:
figures = []

title = f'Classification of Random Forest'
p = draw_activate_image(rf, X, score_type='prediction', use_score=True, resolution=100, height=400, width=400, title=title)
p = scatterplot_2class(X, labels, score=score, p=p)
figures.append(p)

title = f'Variance of prediction p(1-p)'
p = draw_activate_image(rf, X, score_type='var', use_score=True, resolution=100, height=400, width=400, title=title)
p = scatterplot_2class(X, labels, score=score, p=p, colormap=['grey', 'grey'])
figures.append(p)

gp = gridplot([figures[:2]])
# _ = save(gp, './figures/random_forest_moon_variance.html')